In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import sys
from pathlib import Path
import anndata as ad
import scanpy as sc
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import seaborn as sns
from matplotlib import pyplot as plt

sys.path.insert(0, "/home/ylu/project")
from utils import *

sys.path.insert(0, "../../scripts/methods/")
from my_moscot import moscot_align, moscot_align_ref
import time

os.system('/usr/bin/Xvfb :99 -screen 0 1024x768x24 &')
os.environ['DISPLAY'] = ':99'

%load_ext autoreload
%autoreload 2

(EE) 
Fatal server error:
(EE) Server is already active for display 99
	If this server is no longer running, remove /tmp/.X99-lock
	and start again.
(EE) 


In [2]:
data_folder = "./results/split_slices/"
results_folder = "./results/Moscot/"
figures_folder = "./results/figures/Moscot"
Path(results_folder).mkdir(parents=True, exist_ok=True)
Path(figures_folder).mkdir(parents=True, exist_ok=True)

In [3]:
## load the data
from tqdm import tqdm
slices = []
for i in tqdm(range(59)):
    slices.append(ad.read_h5ad(os.path.join(data_folder, f"slice_{i}.h5ad")))

100%|███████████████████████████████████████████████████████████████████| 59/59 [03:00<00:00,  3.05s/it]


In [4]:
spatial_key = "spatial"
anno_key = "class"
palette = slices[0].uns[f"{anno_key}_palette"]

In [5]:
## Rotate the data
rotate_key = "spatial_rot"
rotations = np.load("./results/random_rotations.npy", allow_pickle=True)
for i in range(len(slices)):
    slices[i].obsm[rotate_key] = slices[i].obsm[spatial_key][:,:2].copy()
    mean = np.mean(slices[i].obsm[rotate_key], axis=0)
    slices[i].obsm[rotate_key] = slices[i].obsm[rotate_key] - mean
    slices[i].obsm[rotate_key] = slices[i].obsm[rotate_key] @ rotations[i].T + mean
    

In [6]:
spatial_key = "spatial_rot"
key_added = "aligned_spatial"

In [7]:
## Perform the Moscot alignment
sampling_num = 20000
# for i in tqdm(range(len(slices)-1)):
for i in tqdm(range(41, len(slices)-1)):
    slice1, slice2 = slices[i].copy(), slices[i+1].copy()
    sampline_idx1 = np.random.choice(slice1.shape[0], sampling_num, replace=False) if slice1.shape[0] > sampling_num else np.arange(slice1.shape[0])
    sampline_idx2 = np.random.choice(slice2.shape[0], sampling_num, replace=False) if slice2.shape[0] > sampling_num else np.arange(slice2.shape[0])
    slice1 = slice1[sampline_idx1,:]
    slice2 = slice2[sampline_idx2,:]
    time_start = time.time()
    align_slices, pis = moscot_align(
        models=[slice1, slice2],
        spatial_key=spatial_key,
        key_added=key_added,
        device="cuda",
        verbose=False,
        alpha=0.5,
    )
    time_end = time.time()
    matches = [np.arange(pis[0].shape[0]), np.argmax(pis[0], axis=1)]
    R, t=solve_RT_by_correspondence(align_slices[1].obsm[key_added], align_slices[1].obsm[spatial_key])
    alignment_results = {'sampling_idx1': sampline_idx1, 'sampline_idx2': sampline_idx2, 'R': R, 't': t, 'matches': matches, 'time': time_end - time_start}
    np.save(os.path.join(results_folder, f"slice_{i}_{i+1}_sampling_{sampling_num}.npy"), alignment_results, allow_pickle=True)

  0%|                                                                            | 0/17 [00:00<?, ?it/s]/home/ylu/project/Benchmark_alignment_methods/notebooks/AllenMouseCellAtlas/../../scripts/methods/my_moscot.py:56: ImplicitModificationWarning: Setting element `.obsm['aligned_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


INFO     Computing pca with `n_comps=30` for `xy` using `adata.X`                                                  
INFO     Normalizing spatial coordinates of `x`.                                                                   
INFO     Normalizing spatial coordinates of `y`.                                                                   
INFO     Solving `1` problems                                                                                      
INFO     Solving problem OTProblem[stage='prepared', shape=(20000, 20000)].                                        


2024-07-02 05:36:40.648796: W external/xla/xla/service/gpu/nvptx_compiler.cc:760] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.4.131). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.
  6%|████                                                                | 1/17 [00:37<09:58, 37.39s/it]/home/ylu/project/Benchmark_alignment_methods/notebooks/AllenMouseCellAtlas/../../scripts/methods/my_moscot.py:56: ImplicitModificationWarning: Setting element `.obsm['aligned_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


INFO     Computing pca with `n_comps=30` for `xy` using `adata.X`                                                  
INFO     Normalizing spatial coordinates of `x`.                                                                   
INFO     Normalizing spatial coordinates of `y`.                                                                   
INFO     Solving `1` problems                                                                                      
INFO     Solving problem OTProblem[stage='prepared', shape=(20000, 20000)].                                        


 12%|████████                                                            | 2/17 [01:11<08:50, 35.35s/it]/home/ylu/project/Benchmark_alignment_methods/notebooks/AllenMouseCellAtlas/../../scripts/methods/my_moscot.py:56: ImplicitModificationWarning: Setting element `.obsm['aligned_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


...

 94%|███████████████████████████████████████████████████████████████    | 16/17 [09:34<00:43, 43.59s/it]

INFO     Computing pca with `n_comps=30` for `xy` using `adata.X`                                                  


/home/ylu/project/Benchmark_alignment_methods/notebooks/AllenMouseCellAtlas/../../scripts/methods/my_moscot.py:56: ImplicitModificationWarning: Setting element `.obsm['aligned_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


INFO     Normalizing spatial coordinates of `x`.                                                                   
INFO     Normalizing spatial coordinates of `y`.                                                                   
INFO     Solving `1` problems                                                                                      
INFO     Solving problem OTProblem[stage='prepared', shape=(20000, 13963)].                                        


100%|███████████████████████████████████████████████████████████████████| 17/17 [10:15<00:00, 36.23s/it]
